# Linear Regression Model

## Dependencies + Loading in Data

In [1]:
!pip install statsmodels
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
"""
Variable Descriptions

legId: An identifier for the flight.
searchDate: The date (YYYY-MM-DD) on which this entry was taken from Expedia.
flightDate: The date (YYYY-MM-DD) of the flight.
startingAirport: Three-character IATA airport code for the initial location.
destinationAirport: Three-character IATA airport code for the arrival location.
fareBasisCode: The fare basis code.
travelDuration: The travel duration in hours and minutes.
elapsedDays: The number of elapsed days (usually 0).
isBasicEconomy: Boolean for whether the ticket is for basic economy.
isRefundable: Boolean for whether the ticket is refundable.
isNonStop: Boolean for whether the flight is non-stop.
baseFare: The price of the ticket (in USD).
totalFare: The price of the ticket (in USD) including taxes and other fees.
seatsRemaining: Integer for the number of seats remaining.
totalTravelDistance: The total travel distance in miles. This data is sometimes missing.
segmentsDepartureTimeEpochSeconds: String containing the departure time (Unix time) for each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsDepartureTimeRaw: String containing the departure time (ISO 8601 format: YYYY-MM-DDThh:mm:ss.000±[hh]:00) for each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsArrivalTimeEpochSeconds: String containing the arrival time (Unix time) for each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsArrivalTimeRaw: String containing the arrival time (ISO 8601 format: YYYY-MM-DDThh:mm:ss.000±[hh]:00) for each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsArrivalAirportCode: String containing the IATA airport code for the arrival location for each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsDepartureAirportCode: String containing the IATA airport code for the departure location for each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsAirlineName: String containing the name of the airline that services each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsAirlineCode: String containing the two-letter airline code that services each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsEquipmentDescription: String containing the type of airplane used for each leg of the trip (e.g. "Airbus A321" or "Boeing 737-800"). The entries for each of the legs are separated by '||'.
segmentsDurationInSeconds: String containing the duration of the flight (in seconds) for each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsDistance: String containing the distance traveled (in miles) for each leg of the trip. The entries for each of the legs are separated by '||'.
segmentsCabinCode: String containing the cabin for each leg of the trip (e.g. "coach"). The entries for each of the legs are separated by '||'.
"""
data = pd.read_csv("/work/kaggle_flight_prices/ca_sample_data.csv")
data.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,510f3a1255b9843a8eae48032b191faf,2022-04-17,2022-04-22,OAK,LAX,YH0OASMR,PT7H11M,0,False,False,...,1650678600||1650697140,2022-04-22T18:50:00.000-07:00||2022-04-22T23:5...,SEA||ONT,OAK||SEA,Alaska Airlines||Alaska Airlines,AS||AS,Embraer 175||Airbus A320,7320||8940,672||956,coach||coach
1,5f7a29384cea410317ca308d2e065059,2022-04-17,2022-05-06,SFO,BOS,E0AJZNN1,PT8H29M,1,False,False,...,1651919400||1651929660,2022-05-07T06:30:00.000-04:00||2022-05-07T09:2...,JFK||BOS,SFO||JFK,JetBlue Airways||JetBlue Airways,B6||B6,Airbus A319-321||AIRBUS INDUSTRIE A321 SHARKLETS,20280||4560,2566||185,coach||coach
2,a5e9d9b01627d1e1c54d6b6cbf143945,2022-04-18,2022-04-30,OAK,DTW,QA3OA0MC,PT10H12M,0,False,False,...,1651345740||1651367940||1651376220,2022-04-30T13:09:00.000-06:00||2022-04-30T20:1...,SLC||ORD||DTW,OAK||SLC||ORD,Delta||United||United,DL||UA||UA,Airbus A220-100||Embraer 175 (Enhanced Winglet...,6240||11940||5220,588||1251||240,coach||coach||coach
3,b8714828ff605dfc10a9c511917f1ee8,2022-04-16,2022-04-26,SFO,MIA,R7AZZNN3,PT10H48M,1,False,False,...,1651053360||1651072080,2022-04-27T05:56:00.000-04:00||2022-04-27T11:0...,JFK||MIA,SFO||JFK,JetBlue Airways||JetBlue Airways,B6||B6,Airbus A319-321||Boeing 737 MAX 8,20160||11580,2566||1104,coach||coach
4,80e97c74e379451453a1151b70aaf371,2022-04-17,2022-05-14,LAX,ORD,WAA7OWEN,PT4H4M,1,False,False,...,1652612580,2022-05-15T06:03:00.000-05:00,ORD,LAX,United,UA,Boeing 737-900,14640,1745,coach


## Data Cleaning

In [3]:
# Preprocessing data (part 1)
filtered_data = data[data['segmentsDepartureTimeEpochSeconds'].str.count('\|\|') == 0]
filtered_data['bookingClassCode'] = filtered_data['fareBasisCode'].str[0]
filtered_data["searchDate"] = pd.to_datetime(filtered_data["searchDate"])
filtered_data['flightDate'] = pd.to_datetime(filtered_data['flightDate'])
filtered_data["daysTillFlight"] = (filtered_data["flightDate"] - filtered_data["searchDate"]).dt.days

/tmp/ipykernel_255/3274339856.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['bookingClassCode'] = filtered_data['fareBasisCode'].str[0]
/tmp/ipykernel_255/3274339856.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["searchDate"] = pd.to_datetime(filtered_data["searchDate"])
/tmp/ipykernel_255/3274339856.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [4]:
# Preprocessing data (part 2)
filtered_data = filtered_data.copy()

filtered_data["segmentsDepartureTimeRaw"] = pd.to_datetime(filtered_data["segmentsDepartureTimeRaw"], errors='coerce')
filtered_data.dropna(subset=["segmentsDepartureTimeRaw"], inplace=True)
filtered_data["segmentsDepartureTimeRaw"] = pd.to_datetime(filtered_data["segmentsDepartureTimeRaw"], errors='coerce')
filtered_data.dropna(subset=["segmentsDepartureTimeRaw"], inplace=True)
filtered_data["departureDayOfWeek"] = filtered_data["segmentsDepartureTimeRaw"].dt.dayofweek
filtered_data["departureHour"] = filtered_data["segmentsDepartureTimeRaw"].dt.hour

filtered_data["segmentsArrivalTimeRaw"] = pd.to_datetime(filtered_data["segmentsArrivalTimeRaw"], errors='coerce')
filtered_data.dropna(subset=["segmentsArrivalTimeRaw"], inplace=True)
filtered_data["segmentsArrivalTimeRaw"] = pd.to_datetime(filtered_data["segmentsArrivalTimeRaw"], errors='coerce')
filtered_data.dropna(subset=["segmentsArrivalTimeRaw"], inplace=True)
filtered_data["arrivalDayOfWeek"] = filtered_data["segmentsArrivalTimeRaw"].dt.dayofweek
filtered_data["arrivalHour"] = filtered_data["segmentsArrivalTimeRaw"].dt.hour

/tmp/ipykernel_255/2557190885.py:4: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  filtered_data["segmentsDepartureTimeRaw"] = pd.to_datetime(filtered_data["segmentsDepartureTimeRaw"], errors='coerce')
/tmp/ipykernel_255/2557190885.py:11: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  filtered_data["segmentsArrivalTimeRaw"] = pd.to_datetime(filtered_data["segmentsArrivalTimeRaw"], errors='coerce')


In [5]:
# Preprocessing data (part 3)
filtered_data['segmentsDurationInSeconds'] = filtered_data['segmentsDurationInSeconds'].astype(int)
filtered_data.dropna(subset=["segmentsDistance"], inplace=True)
filtered_data['segmentsDistance'] = filtered_data['segmentsDistance'].astype(int)
filtered_data = filtered_data.drop(columns=['legId', 'totalFare', 'fareBasisCode', 'travelDuration', 'segmentsDepartureTimeRaw', 'segmentsArrivalTimeRaw', 'segmentsDepartureTimeEpochSeconds', 'segmentsArrivalTimeEpochSeconds', 'segmentsAirlineName'])

In [6]:
# Displaying the final filtered dataframe

filtered_data.head()

,searchDate,flightDate,startingAirport,destinationAirport,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,seatsRemaining,...,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode,bookingClassCode,daysTillFlight,departureDayOfWeek,departureHour,arrivalDayOfWeek,arrivalHour
4,2022-04-17,2022-05-14,LAX,ORD,1,False,False,True,236.28,9,...,Boeing 737-900,14640,1745,coach,W,27,5,23,6,6
70,2022-04-16,2022-04-19,LAX,ORD,0,False,False,True,305.12,6,...,Boeing 757-300,14880,1745,coach,Q,3,1,10,1,16
109,2022-04-18,2022-04-19,LAX,ORD,0,False,False,True,322.79,5,...,Boeing 737-800,14700,1745,coach,M,1,1,13,1,19
170,2022-04-17,2022-04-25,LAX,ORD,0,False,False,True,459.53,7,...,Boeing 737-800,14640,1745,coach,H,8,0,7,0,13
239,2022-04-18,2022-04-23,LAX,DFW,0,False,False,True,252.09,7,...,Airbus A321,10860,1238,coach,L,5,5,18,5,23


## Dummy Encoding Data + Train/Test Split

In [7]:
# Dummy encoding the data (and getting rid of time series columns)
encoded_data = pd.get_dummies(filtered_data.drop(columns=['searchDate', 'flightDate']))
encoded_data = encoded_data.astype('float64')

In [8]:
# Train test split
y = encoded_data["baseFare"]
X = encoded_data.copy().drop(columns=["baseFare"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
# Adding intercept
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

## Basic LinReg Model

In [10]:
basic_linreg = sm.OLS(y_train, X_train).fit()
basic_linreg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               baseFare   R-squared:                       0.814
Model:                            OLS   Adj. R-squared:                  0.814
Method:                 Least Squares   F-statistic:                     1090.
Date:                Wed, 11 Dec 2024   Prob (F-statistic):               0.00
Time:                        23:05:41   Log-Likelihood:                -99306.
No. Observations:               17940   AIC:                         1.988e+05
Df Residuals:                   17867   BIC:                         1.993e+05
Df Model:                          72                                         
Covariance Type:            nonrobust                                         
================================================================================================================================
                                                                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------------
elapsedDays                                                      0.0103     33.094      0.000      1.000     -64.857      64.877
isBasicEconomy                                                 -61.2780      1.570    -39.022      0.000     -64.356     -58.200
isRefundable                                                 -1.114e-10   8.15e-12    -13.658      0.000   -1.27e-10   -9.54e-11
isNonStop                                                      228.3056     17.232     13.249      0.000     194.529     262.082
seatsRemaining                                                   0.3563      0.245      1.452      0.147      -0.125       0.837
totalTravelDistance                                             -0.1664      0.011    -15.034      0.000      -0.188      -0.145
segmentsDurationInSeconds                                        0.0517      0.002     29.564      0.000       0.048       0.055
segmentsDistance                                                -0.1664      0.011    -15.034      0.000      -0.188      -0.145
daysTillFlight                                                  -0.3216      0.029    -10.953      0.000      -0.379      -0.264
departureDayOfWeek                                               1.8902      0.489      3.866      0.000       0.932       2.848
departureHour                                                   -0.4684      1.414     -0.331      0.740      -3.240       2.303
arrivalDayOfWeek                                                 0.2406      0.489      0.492      0.623      -0.717       1.199
arrivalHour                                                      0.7791      1.408      0.553      0.580      -1.980       3.538
startingAirport_LAX                                            124.3218      8.945     13.899      0.000     106.789     141.855
startingAirport_SFO                                            103.9838      8.330     12.483      0.000      87.656     120.311
destinationAirport_ATL                                          -1.7037     28.811     -0.059      0.953     -58.177      54.769
destinationAirport_BOS                                          25.4389     14.513      1.753      0.080      -3.008      53.886
destinationAirport_CLT                                          56.0466     11.995      4.672      0.000      32.535      79.558
destinationAirport_DEN                                         -13.0280     44.369     -0.294      0.769     -99.995      73.939
destinationAirport_DFW                                           2.6820     19.362      0.139      0.890     -35.270      40.634
destinationAirport_DTW                                          61.0921     17.442      3.503      0.000      26.904

Based on the p-values from our model summary of the basic linear regression model, we can see that some of the continuous variables are not significant (> 0.05). We can remove the following continuous features from the model:
- `seatsRemaining`
- `arrivalDayOfWeek`
- `arrivalHour`

In [11]:
# Dropping insignificant continuous features
features_to_remove_1 = ["seatsRemaining", "arrivalDayOfWeek", "arrivalHour"] # Manually identified
X_train_filtered_1 = X_train.drop(columns=features_to_remove_1)

## Filtered LinReg Model (Iteration 1)

In [12]:
linreg_filtered_1 = sm.OLS(y_train, X_train_filtered_1).fit()
linreg_filtered_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               baseFare   R-squared:                       0.814
Model:                            OLS   Adj. R-squared:                  0.814
Method:                 Least Squares   F-statistic:                     1137.
Date:                Wed, 11 Dec 2024   Prob (F-statistic):               0.00
Time:                        23:05:43   Log-Likelihood:                -99307.
No. Observations:               17940   AIC:                         1.988e+05
Df Residuals:                   17870   BIC:                         1.993e+05
Df Model:                          69                                         
Covariance Type:            nonrobust                                         
================================================================================================================================
                                                                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------------
elapsedDays                                                    -18.2408      2.071     -8.809      0.000     -22.299     -14.182
isBasicEconomy                                                 -60.8402      1.537    -39.581      0.000     -63.853     -57.827
isRefundable                                                 -4.261e-09   3.15e-10    -13.513      0.000   -4.88e-09   -3.64e-09
isNonStop                                                      230.2774     16.897     13.628      0.000     197.158     263.397
totalTravelDistance                                             -0.1671      0.011    -15.315      0.000      -0.188      -0.146
segmentsDurationInSeconds                                        0.0520      0.002     30.997      0.000       0.049       0.055
segmentsDistance                                                -0.1671      0.011    -15.315      0.000      -0.188      -0.146
daysTillFlight                                                  -0.3142      0.029    -10.873      0.000      -0.371      -0.258
departureDayOfWeek                                               2.0927      0.231      9.064      0.000       1.640       2.545
departureHour                                                    0.3150      0.127      2.476      0.013       0.066       0.564
startingAirport_LAX                                            125.4479      8.757     14.325      0.000     108.283     142.613
startingAirport_SFO                                            104.8294      8.182     12.812      0.000      88.791     120.868
destinationAirport_ATL                                          -1.1017     28.808     -0.038      0.969     -57.568      55.364
destinationAirport_BOS                                          25.6498     14.479      1.772      0.076      -2.730      54.030
destinationAirport_CLT                                          56.3261     11.989      4.698      0.000      32.826      79.827
destinationAirport_DEN                                         -14.5354     44.351     -0.328      0.743    -101.468      72.397
destinationAirport_DFW                                           2.9013     19.345      0.150      0.881     -35.016      40.819
destinationAirport_DTW                                          61.1390     17.438      3.506      0.000      26.960      95.318
destinationAirport_EWR                                          -0.8023     15.854     -0.051      0.960     -31.877      30.273
destinationAirport_IAD                                          29.8283     17.636      1.691      0.091      -4.740      64.396
destinationAirport_JFK                                          22.7780     10.126      2.250      0.024       2.931

Based on the p-values from our model summary of the filtered linear regression model, we can see that some of the categorical variables are not significant (> 0.05). We can remove them from the model

In [13]:
# Grab features with p-values > 0.05
features_to_remove_2 = linreg_filtered_1.pvalues[linreg_filtered_1.pvalues > 0.05].index.tolist()

# Dropping insignificant continuous features
X_train_filtered_2 = X_train.drop(columns=features_to_remove_1+features_to_remove_2)

## Filtered LinReg Model (Iteration 2)

In [14]:
linreg_filtered_2 = sm.OLS(y_train, X_train_filtered_2).fit()
linreg_filtered_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               baseFare   R-squared:                       0.814
Model:                            OLS   Adj. R-squared:                  0.814
Method:                 Least Squares   F-statistic:                     1705.
Date:                Wed, 11 Dec 2024   Prob (F-statistic):               0.00
Time:                        23:05:43   Log-Likelihood:                -99318.
No. Observations:               17940   AIC:                         1.987e+05
Df Residuals:                   17893   BIC:                         1.991e+05
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
elapsedDays                                     -17.8794      2.031     -8.805      0.000     -21.860     -13.899
isBasicEconomy                                  -60.7594      1.521    -39.957      0.000     -63.740     -57.779
isRefundable                                   -1.22e-09   7.63e-11    -15.992      0.000   -1.37e-09   -1.07e-09
isNonStop                                       191.8063     16.187     11.850      0.000     160.079     223.534
totalTravelDistance                              -0.1566      0.007    -21.202      0.000      -0.171      -0.142
segmentsDurationInSeconds                         0.0515      0.002     31.521      0.000       0.048       0.055
segmentsDistance                                 -0.1566      0.007    -21.202      0.000      -0.171      -0.142
daysTillFlight                                   -0.3102      0.029    -10.755      0.000      -0.367      -0.254
departureDayOfWeek                                2.0923      0.231      9.069      0.000       1.640       2.545
departureHour                                     0.3021      0.126      2.393      0.017       0.055       0.549
startingAirport_LAX                             104.5043      8.882     11.766      0.000      87.095     121.914
startingAirport_SFO                              87.3020      7.345     11.886      0.000      72.906     101.698
destinationAirport_CLT                           49.2846     12.435      3.964      0.000      24.912      73.658
destinationAirport_DTW                           56.3147     18.426      3.056      0.002      20.198      92.431
destinationAirport_JFK                           18.1686      9.227      1.969      0.049       0.083      36.254
destinationAirport_ORD                           13.0894      2.411      5.428      0.000       8.363      17.816
segmentsArrivalAirportCode_CLT                   49.2846     12.435      3.964      0.000      24.912      73.658
segmentsArrivalAirportCode_DTW                   56.3147     18.426      3.056      0.002      20.198      92.431
segmentsArrivalAirportCode_JFK                   18.1686      9.227      1.969      0.049       0.083      36.254
segmentsArrivalAirportCode_ORD                   13.0894      2.411      5.428      0.000       8.363      17.816
segmentsDepartureAirportCode_LAX                 28.2597      4.805      5.882      0.000      18.842      37.677
segmentsDepartureAirportCode_ONT                 76.2445      4.291     17.770      0.000      67.835      84.654
segmentsDepartureAirportCode_SFO                 87.3020      7.345     11.886      0.000      72.906     101.698
segmentsAirlineCode_AA                          187.2474     22.001      8.511      0.000     144.124     230.371
segme

We can see that more categorical variables are not significant (> 0.05). We can remove them from the model one last time!

In [15]:
# Grab features with p-values > 0.05 (again)
features_to_remove_3 = linreg_filtered_2.pvalues[linreg_filtered_2.pvalues > 0.05].index.tolist()

# Dropping insignificant categorical features (again)
X_train_filtered_3 = X_train.drop(columns=features_to_remove_1+features_to_remove_2+features_to_remove_3)

## Filtered LinReg Model (Iteration 3)

In [16]:
linreg_filtered_3 = sm.OLS(y_train, X_train_filtered_3).fit()
linreg_filtered_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               baseFare   R-squared:                       0.814
Model:                            OLS   Adj. R-squared:                  0.814
Method:                 Least Squares   F-statistic:                     1743.
Date:                Wed, 11 Dec 2024   Prob (F-statistic):               0.00
Time:                        23:05:44   Log-Likelihood:                -99319.
No. Observations:               17940   AIC:                         1.987e+05
Df Residuals:                   17894   BIC:                         1.991e+05
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
elapsedDays                                     -17.8326      2.031     -8.782      0.000     -21.813     -13.853
isBasicEconomy                                  -60.7293      1.521    -39.939      0.000     -63.710     -57.749
isRefundable                                  -9.121e-10    5.5e-11    -16.585      0.000   -1.02e-09   -8.04e-10
isNonStop                                       171.7065      9.152     18.761      0.000     153.768     189.645
totalTravelDistance                              -0.1540      0.007    -21.462      0.000      -0.168      -0.140
segmentsDurationInSeconds                         0.0513      0.002     31.495      0.000       0.048       0.054
segmentsDistance                                 -0.1540      0.007    -21.462      0.000      -0.168      -0.140
daysTillFlight                                   -0.3095      0.029    -10.731      0.000      -0.366      -0.253
departureDayOfWeek                                2.0924      0.231      9.069      0.000       1.640       2.545
departureHour                                     0.2983      0.126      2.363      0.018       0.051       0.546
startingAirport_LAX                              93.2521      4.799     19.433      0.000      83.846     102.658
startingAirport_SFO                              78.4544      4.405     17.809      0.000      69.820      87.089
destinationAirport_CLT                           47.4881     12.378      3.837      0.000      23.227      71.749
destinationAirport_DTW                           54.8135     18.400      2.979      0.003      18.748      90.879
destinationAirport_JFK                           21.5458      8.950      2.407      0.016       4.002      39.090
destinationAirport_ORD                           12.1832      2.335      5.217      0.000       7.606      16.760
segmentsArrivalAirportCode_CLT                   47.4881     12.378      3.837      0.000      23.227      71.749
segmentsArrivalAirportCode_DTW                   54.8135     18.400      2.979      0.003      18.748      90.879
segmentsArrivalAirportCode_JFK                   21.5458      8.950      2.407      0.016       4.002      39.090
segmentsArrivalAirportCode_ORD                   12.1832      2.335      5.217      0.000       7.606      16.760
segmentsDepartureAirportCode_LAX                 22.5390      2.940      7.666      0.000      16.776      28.302
segmentsDepartureAirportCode_ONT                 70.7131      2.216     31.914      0.000      66.370      75.056
segmentsDepartureAirportCode_SFO                 78.4544      4.405     17.809      0.000      69.820      87.089
segmentsAirlineCode_AA                          220.0121      3.219     68.347      0.000     213.702     226.322
segme

## Performance Metrics + Analysis

In [17]:
def OSR2(model, X_test, y_test, y_train):
    y_pred = model.predict(X_test)
    SSE = np.sum((y_test - y_pred)**2)
    SST = np.sum((y_test - np.mean(y_test))**2)
    return (1 - SSE/SST)

X_test_filtered = X_test.drop(columns=features_to_remove_1+features_to_remove_2+features_to_remove_3)
print(f"OSR2: {OSR2(linreg_filtered_3, X_test_filtered, y_test, y_train)}")

OSR2: 0.817788167875065


In [18]:
def RMSE(model, X_test, y_test, y_train):
    y_pred = model.predict(X_test)
    return np.sqrt(np.mean((y_test - y_pred) ** 2))

X_test_filtered = X_test.drop(columns=features_to_remove_1+features_to_remove_2+features_to_remove_3)
print(f"RMSE: {RMSE(linreg_filtered_3, X_test_filtered, y_test, y_train)}")

RMSE: 62.158129548633056


In [19]:
print("Number of variables in final linear regression model:", len(X_train_filtered_3.columns))

Number of variables in final linear regression model: 56


In [20]:
print("Kept features in final linear regression model:\n", X_train_filtered_3.columns)

Kept features in final linear regression model:
 Index(['elapsedDays', 'isBasicEconomy', 'isRefundable', 'isNonStop',
       'totalTravelDistance', 'segmentsDurationInSeconds', 'segmentsDistance',
       'daysTillFlight', 'departureDayOfWeek', 'departureHour',
       'startingAirport_LAX', 'startingAirport_SFO', 'destinationAirport_CLT',
       'destinationAirport_DTW', 'destinationAirport_JFK',
       'destinationAirport_ORD', 'segmentsArrivalAirportCode_CLT',
       'segmentsArrivalAirportCode_DTW', 'segmentsArrivalAirportCode_JFK',
       'segmentsArrivalAirportCode_ORD', 'segmentsDepartureAirportCode_LAX',
       'segmentsDepartureAirportCode_ONT', 'segmentsDepartureAirportCode_SFO',
       'segmentsAirlineCode_AA', 'segmentsAirlineCode_AS',
       'segmentsAirlineCode_DL', 'segmentsAirlineCode_UA',
       'segmentsEquipmentDescription_Airbus A319',
       'segmentsEquipmentDescription_BOEING 787-9',
       'segmentsEquipmentDescription_Boeing 737 MAX 9',
       'segmentsEquipmentD

In [21]:
print("Features removed in final linear regression model:\n", features_to_remove_1+features_to_remove_2+features_to_remove_3)

Features removed in final linear regression model:
 ['seatsRemaining', 'arrivalDayOfWeek', 'arrivalHour', 'destinationAirport_ATL', 'destinationAirport_BOS', 'destinationAirport_DEN', 'destinationAirport_DFW', 'destinationAirport_EWR', 'destinationAirport_IAD', 'destinationAirport_MIA', 'destinationAirport_PHL', 'segmentsArrivalAirportCode_ATL', 'segmentsArrivalAirportCode_BOS', 'segmentsArrivalAirportCode_DAL', 'segmentsArrivalAirportCode_DFW', 'segmentsArrivalAirportCode_EWR', 'segmentsArrivalAirportCode_IAD', 'segmentsArrivalAirportCode_MIA', 'segmentsArrivalAirportCode_PHL', 'segmentsAirlineCode_B6', 'segmentsEquipmentDescription_AIRBUS INDUSTRIE A321 SHARKLETS', 'segmentsEquipmentDescription_Airbus A319-321', 'segmentsEquipmentDescription_Airbus A320', 'segmentsEquipmentDescription_Airbus A321', 'segmentsEquipmentDescription_BOEING 777-300ER', 'segmentsEquipmentDescription_Boeing 737 MAX 8', 'segmentsEquipmentDescription_Boeing 737-800', 'segmentsEquipmentDescription_Boeing 757-30

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=36e46b6c-b77a-4f8c-a857-c3dc2e354e3d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>